In [1]:
!git clone https://github.com/Reennon/ua-gec-lora.git
!cd ua-gec-lora && pip install -r requirements.txt
!pwd && ls -a
# Install additional libs
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/trl.git@7630f877f91c556d9e5a3baa4b6e2894d90ff84c
!pip install ua_gec
!pip install datasets==2.16.0
!pip install nltk
!pip install wandb -q -U
# CD into the project directory
%cd ua-gec-lora
!git pull origin "feature/fine-tuning-research"
!git status

Cloning into 'ua-gec-lora'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 94 (delta 26), reused 71 (delta 13), pack-reused 0
Unpacking objects: 100% (94/94), 100.75 KiB | 3.15 MiB/s, done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.1 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / - \ done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoModelForCausalLM, pipeline, Conversation, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, TaskType, PeftModel, get_peft_model, prepare_model_for_kbit_training
from src.packages.constants.error_constants import ErrorConstants
from src.packages.prompts.instruction_tuning_gec_prompts import InstructionTuningGecPrompts
from ua_gec import Corpus
from langchain.prompts import PromptTemplate
import torch
import nltk
import re

nltk.download('punkt')  # Download the necessary resources for sentence tokenization

from nltk.tokenize import sent_tokenize

2024-03-22 18:17:16.145515: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 18:17:16.145640: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 18:17:16.287017: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2" #"/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1" 

In [5]:
fine_tuned_model_name = 'andrian-kr/mistral-7b-ua-gec'

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant= True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16, 
    device_map={'':torch.cuda.current_device()},)

model = PeftModel.from_pretrained(base_model, fine_tuned_model_name)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/23.1M [00:00<?, ?B/s]

In [7]:
template = """[INST] Given a text ("ORIGINAL_TEXT") in Ukrainian with potential errors, correct them to fulfill the GEC (Grammar Error Correction) Task for Ukrainian Language.
Consider the provided set of error types ("ERROR_TYPES"):
{error_types}
When you identify an error ("ERROR") in the text, correct it according to the format:
("ERROR") => ("CORRECTION")
Where ("CORRECTION") Specifies how the error should be corrected without providing explicit reasoning for the change.
The resulting text ("FIXED_TEXT") should be error-free, maintaining the original information's semantics.
Focus solely on correcting Ukrainian language errors.
Ensure that the corrected text doesn't include original errors, additional text, comments, or parts of these instructions.
Do not translate the text; perform grammar correction only for Ukrainian language.
Keep original information intact, prioritizing the semantics of original text being intact.
Do not repeat the sentences in the corrected text.
Do not separately provide information about the errors and correction. Only corrected text with the specified format.

ORIGINAL_TEXT: {query}
FIXED_TEXT:
[/INST]"""

it_prompt = PromptTemplate(
    template=template,
    input_variables=['query', 'error_types']
)

In [8]:
max_sentences = None #5

In [9]:
corpus = list(Corpus(partition="test", annotation_layer="gec-only"))
doc = corpus[0]
# for doc in corpus:
#     print("\n---Source starts:---\n")
#     print(doc.source)         # "I likes it."
#     print("\n---Source ends:---\n")
#     print("\n---Target starts:---\n")
#     print(doc.target)         # "I like it."
#     print("\n---Target ends---\n")
#     print("\n---Annotation starts:---\n")
#     print(str(doc.annotated)[:1200])      # <AnnotatedText("I {likes=>like} it.")
#     print(doc.meta.region)    # "Київська"
#     print("\n---Annotation ends")
print(doc.doc_id)
print(doc.source)
print("\n---Prompt for training:")
source = "".join(sent_tokenize(doc.source)[:max_sentences] if max_sentences else sent_tokenize(doc.source))
target = "".join(sent_tokenize(doc.target))
prompt = it_prompt.format_prompt(
    query=doc.source,
    error_types=ErrorConstants.ERROR_TYPES
).to_string()
#     sample_text = ' '.join(prompt.split())
#     target_text = ' '.join(target.split())
#     sample_text += target_text
print(prompt)

0002
Наступного ранку рівно о одинадцятій годині, коли я сидів сам, дядько Том шаштався в готелі і попросив у лікаря підійти і побачити Джанге Банк, хто, воно здавалось, це був майор і дуже хворий чоловік.
"Я не є доктор" - сказав я: "Чому вам не піти до лікаря?".
"Босс" - сказав він: "Доктор Хоскінс має проїхати 20 миль по країні, щоб побачити хворих персон. Він є єдиний лікар в місті і Масса банки сильно погано обурудувані. Він відправив мені, щоб спитав чи погоджуйтеся хоч прийти".
"Як чоловік до чоловіка" - я сказав: "Я буду йти і пошукаю його пізніше". Отже, Я підняв пляшку "Recurrection Bitters" з мого ранця і пішов на схил до майорівського особняку, будинок в місті, з розкішною кришою і два виготовлені залізних собак на газоні.


---Prompt for training:
[INST] Given a text ("ORIGINAL_TEXT") in Ukrainian with potential errors, correct them to fulfill the GEC (Grammar Error Correction) Task for Ukrainian Language.
Consider the provided set of error types ("ERROR_TYPES"):
['Fluency

In [10]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    padding_side='left',
    trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [11]:
prompt_len = len(tokenizer.tokenize(prompt))
source_len = len(tokenizer.tokenize(doc.source))
max_correction_addtional_tokens = 0.2
max_new_tokens = int( len(tokenizer.tokenize(doc.source)) * 1.2)
# max_tokens = int(prompt_len * 1.1)
max_tokens = prompt_len + source_len

print(f"""
Prompt len: {prompt_len}
MAx tokens: {max_tokens}
Max token difference because of corrections: {max_correction_addtional_tokens}
Max new tokens len (output without input): {max_new_tokens}
""")


Prompt len: 618
MAx tokens: 955
Max token difference because of corrections: 0.2
Max new tokens len (output without input): 404



In [12]:
# Fix padding token for Mistral and Phi-2 models
tokenizer.pad_token = "[PAD]"

In [13]:
model_inputs = tokenizer(
    prompt, 
    max_length=max_tokens, 
    padding="max_length", 
    truncation=True, 
    return_tensors="pt"
)
model_inputs

{'input_ids': tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [14]:
model = model.eval()

In [15]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [16]:
response = model.generate(
    input_ids=model_inputs["input_ids"].to(device),
    attention_mask=model_inputs["attention_mask"].to(device),
    max_new_tokens=max_tokens,
    pad_token_id=tokenizer.eos_token_id
)
response

tensor([[    0,     0,     0,  ...,  2077, 28723,     2]], device='cuda:0')

In [17]:
decoded_outputs = tokenizer.batch_decode(response.detach().cpu().numpy(), skip_special_tokens=True)
text = decoded_outputs[0][len(prompt):]
print(text)

Наступного ранку рівно о одинадцятій годині, коли я сидів сам, дядько Том шаштався в готелі і попросив у лікаря підійти і побачити Джанге Банк, хто, воно здавалось, це був майор і дуже хворий чоловік. "Я не є доктором" — сказав я.— Чому вам не піти до лікаря?". "Босс", — сказав він,— "Доктор Хоскінс має проїхати 20 миль по країні, щоб побачити хворих персон. Він є єдиний лікар в місті, і Масса банки сильно погано обурудовані. Він відправив мені, щоб спитав чи погоджуйтеся хоч прийти". "Як чоловік до чоловіка" — я сказав.— Я буду йти і пошукаю його пізніше". Отже, я підняв пляшку "Recurrection Bitters" з мого ранця і пішов на схил до майорівського особняка, будинок в місті з розкішною кришою і два виготовлені залізні собаки на газоні.

Виправлені помилки:
— Чому вам не піти до лікаря?— Босс, — сказав він,— Доктор Хоскінс має проїхати 20 миль по країні, щоб побачити хворих персон. Він є єдиний лікар в місті, і Масса банки сильно погано обурудовані. Він відправив мені, щоб спитав чи погод

In [18]:
text_with_sentences = re.sub(r'(?<=[.!?])(?![\'"])', ' ', text)
print("\n".join(sent_tokenize(text_with_sentences)))

Наступного ранку рівно о одинадцятій годині, коли я сидів сам, дядько Том шаштався в готелі і попросив у лікаря підійти і побачити Джанге Банк, хто, воно здавалось, це був майор і дуже хворий чоловік.
"Я не є доктором" — сказав я.
— Чому вам не піти до лікаря?".
"Босс", — сказав він,— "Доктор Хоскінс має проїхати 20 миль по країні, щоб побачити хворих персон.
Він є єдиний лікар в місті, і Масса банки сильно погано обурудовані.
Він відправив мені, щоб спитав чи погоджуйтеся хоч прийти".
"Як чоловік до чоловіка" — я сказав.
— Я буду йти і пошукаю його пізніше".
Отже, я підняв пляшку "Recurrection Bitters" з мого ранця і пішов на схил до майорівського особняка, будинок в місті з розкішною кришою і два виготовлені залізні собаки на газоні.
Виправлені помилки:
— Чому вам не піти до лікаря?
— Босс, — сказав він,— Доктор Хоскінс має проїхати 20 миль по країні, щоб побачити хворих персон.
Він є єдиний лікар в місті, і Масса банки сильно погано обурудовані.
Він відправив мені, щоб спитав чи пог

In [19]:
from difflib import SequenceMatcher
from nltk.tokenize import word_tokenize

def normalize_spaces(text):
    return ' '.join(text.split())

def get_highlighted_changes(text1, text2):
    
    words1 = text1.split()
    words2 = text2.split()

    highlighted_text = []
    index1 = 0
    index2 = 0
    while index2 < len(words2):
        word2 = words2[index2] if index2 < len(words2) else ''
        word1 = words1[index1] if index1 < len(words1) else ''

        # Extract actual words without color codes
        actual_word1 = re.sub(r'\033\[\d+m', '', word1)
        actual_word2 = re.sub(r'\033\[\d+m', '', word2)
        
        if '\033' in word2:
            if '\033' in word1:
                # Both words are highlighted, mark them as orange
                highlighted_text.append('\033[93m\033[1m' + actual_word2 + '\033[0m')
                index1 +=1
            else:
                highlighted_text.append('\033[92m\033[1m' + actual_word2 + '\033[0m')
        else:
            if actual_word1.lower() == actual_word2.lower():
                # No change, just append the word as is
                highlighted_text.append(word2)
                index1 +=1
            else:
                highlighted_text.append('\033[92m\033[1m' + actual_word2 + '\033[0m')

        index2 += 1

    return ' '.join(highlighted_text)

def generate_original_corrected_texts(original_text, corrected_text):
    original_full_text = ''
    corrected_full_text = ''
    highlighted_full_text = ''
    
    original_words = original_text.split()
    corrected_words = corrected_text.split()

    marked_original_text = []
    marked_corrected_text = []

    # Track words from the original text that were removed
    removed_words = set(original_words) - set(corrected_words)

    # Track words from the corrected text that were added
    added_words = set(corrected_words) - set(original_words)

    # Mark removed words in the original text as red
    for word in original_words:
        if word in removed_words:
            marked_original_text.append('\033[91m\033[1m' + word + '\033[0m')
        else:
            marked_original_text.append(word)

    # Mark added words in the corrected text as green
    for word in corrected_words:
        if word in added_words:
            marked_corrected_text.append('\033[92m\033[1m' + word + '\033[0m')
        else:
            marked_corrected_text.append(word)

    marked_original_sentence = ' '.join(marked_original_text)
    marked_corrected_sentence = ' '.join(marked_corrected_text)
    highlighted_comparison = get_highlighted_changes(marked_original_sentence, marked_corrected_sentence)

    original_full_text += marked_original_sentence + '\n'
    corrected_full_text += marked_corrected_sentence + '\n'
    highlighted_full_text += highlighted_comparison + '\n'
        
    return (original_full_text, corrected_full_text, highlighted_full_text)

original = "\n".join(sent_tokenize(doc.source)[:max_sentences])
# text2 = "\n".join(sent_tokenize(text[1:]))

# highlighted_text = highlight_changes(text1, text2)

original_text, corrected_text, highlighted_text = generate_original_corrected_texts(
    original_text=original, 
    corrected_text=text_with_sentences)

print("Original Text:")
print(original_text)
print()

print("Corrected Text:")
print(corrected_text)
print()

print("Changes comparison:")
print(highlighted_text)


Original Text:
Наступного ранку рівно о одинадцятій годині, коли я сидів сам, дядько Том шаштався в готелі і попросив у лікаря підійти і побачити Джанге Банк, хто, воно здавалось, це був майор і дуже хворий чоловік. "Я не є доктор" - сказав я: "Чому вам не піти до лікаря?". "Босс" - сказав він: "Доктор Хоскінс має проїхати 20 миль по країні, щоб побачити хворих персон. Він є єдиний лікар в місті і Масса банки сильно погано обурудувані. Він відправив мені, щоб спитав чи погоджуйтеся хоч прийти". "Як чоловік до чоловіка" - я сказав: "Я буду йти і пошукаю його пізніше". Отже, Я підняв пляшку "Recurrection Bitters" з мого ранця і пішов на схил до майорівського особняку, будинок в місті, з розкішною кришою і два виготовлені залізних собак на газоні.


Corrected Text:
Наступного ранку рівно о одинадцятій годині, коли я сидів сам, дядько Том шаштався в готелі і попросив у лікаря підійти і побачити Джанге Банк, хто, воно здавалось, це був майор і дуже хворий чоловік. "Я не є доктором" — сказав

In [20]:
from difflib import SequenceMatcher
import re

def normalize_spaces(text):
    return ' '.join(text.split())

def highlight_changes(text1, text2):
    # Tokenize the texts into words
    words1 = re.findall(r'\w+|[^\w\s]', text1)
    words2 = re.findall(r'\w+|[^\w\s]', text2)


    # Find the unique words present in both texts
    all_words = set(words1 + words2)

    # Initialize a SequenceMatcher object
    matcher = SequenceMatcher(None, words1, words2)

    # Get the differences
    diff = matcher.get_opcodes()

    highlighted_text = []

    for op, start1, end1, start2, end2 in diff:
        if op == 'equal':
            # No change, just append the words as is
            highlighted_text.extend(words1[start1:end1])
        elif op == 'delete':
            # Word(s) removed, highlight with red
            for word in words1[start1:end1]:
                word = '\u0336'.join(word) + '\u0336'
                highlighted_text.append('\033[91m\033[1m' + word + '\033[0m')
        elif op == 'insert':
            # Word(s) added, highlight with green
            for word in words2[start2:end2]:
                highlighted_text.append('\033[92m\033[1m' + word + '\033[0m')
        elif op == 'replace':
            # Word(s) replaced, highlight with yellow
            for word in words2[start2:end2]:
                highlighted_text.append('\033[93m\033[1m' + word + '\033[0m')

    return ' '.join(highlighted_text)

def generate_original_corrected_texts(original_text, corrected_text, highlighted_comparison):
    # Split the original and corrected texts
    original_words = original_text.split()
    corrected_words = corrected_text.split()

    # Initialize empty lists for marked original and corrected texts
    marked_original_text = []
    marked_corrected_text = []

    # Track words from the original text that were removed
    removed_words = set(original_words) - set(corrected_words)

    # Track words from the corrected text that were added
    added_words = set(corrected_words) - set(original_words)

    # Mark removed words in the original text as red
    for word in original_words:
        if word in removed_words:
            marked_original_text.append('\033[91m\033[1m' + word + '\033[0m')
        else:
            marked_original_text.append(word)

    # Mark added words in the corrected text as green
    for word in corrected_words:
        if word in added_words:
            marked_corrected_text.append('\033[92m\033[1m' + word + '\033[0m')
        else:
            marked_corrected_text.append(word)

    return (' '.join(marked_original_text), ' '.join(marked_corrected_text), highlighted_comparison)

# text1 = normalize_spaces("".join(sent_tokenize(doc.source)[:max_sentences]))
# text2 = normalize_spaces(text[1:])

highlighted_text = highlight_changes(original, text_with_sentences)

original_text, corrected_text, _ = generate_original_corrected_texts(
    original_text=original, 
    corrected_text=text_with_sentences, 
    highlighted_comparison=highlighted_text)

print("Original Text:")
print(original_text)
print()

print("Corrected Text:")
print(corrected_text)
print()

print("Changes comparison:")
print(highlighted_text)

Original Text:
Наступного ранку рівно о одинадцятій годині, коли я сидів сам, дядько Том шаштався в готелі і попросив у лікаря підійти і побачити Джанге Банк, хто, воно здавалось, це був майор і дуже хворий чоловік. "Я не є доктор" - сказав я: "Чому вам не піти до лікаря?". "Босс" - сказав він: "Доктор Хоскінс має проїхати 20 миль по країні, щоб побачити хворих персон. Він є єдиний лікар в місті і Масса банки сильно погано обурудувані. Він відправив мені, щоб спитав чи погоджуйтеся хоч прийти". "Як чоловік до чоловіка" - я сказав: "Я буду йти і пошукаю його пізніше". Отже, Я підняв пляшку "Recurrection Bitters" з мого ранця і пішов на схил до майорівського особняку, будинок в місті, з розкішною кришою і два виготовлені залізних собак на газоні.

Corrected Text:
Наступного ранку рівно о одинадцятій годині, коли я сидів сам, дядько Том шаштався в готелі і попросив у лікаря підійти і побачити Джанге Банк, хто, воно здавалось, це був майор і дуже хворий чоловік. "Я не є доктором" — сказав 

In [21]:
import os

output_folder = os.path.join(os.getcwd(), 'fine-tuned-result')
os.makedirs(output_folder, exist_ok=True)
result_path = os.path.join(output_folder,f'ouput.txt')

In [22]:
def generate_promt(source):
    prompt = it_prompt.format_prompt(
        query=source,
        error_types=ErrorConstants.ERROR_TYPES
    ).to_string()
    
    return prompt

In [23]:
test_data = list(Corpus(partition="test", annotation_layer="gec-only"))

In [24]:
from tqdm import tqdm

max_correction_addtional_tokens = 0.2

# filter duplicate ids
unique_doc_ids = set()

for doc in tqdm(test_data[:1]):
    id_str = doc.doc_id
    
    if id_str in unique_doc_ids:
        continue
        
    unique_doc_ids.add(id_str)
    
    # tokenize input
    prompt = generate_promt(doc.source)
    max_token_length = len(tokenizer.tokenize(prompt)) + len(tokenizer.tokenize(doc.source))
    tokenized_prompt = tokenizer(
        prompt, 
        max_length=max_token_length, 
        padding="max_length", 
        truncation=True, 
        return_tensors="pt"
    )
    
    torch.backends.cuda.enable_mem_efficient_sdp(False)
    torch.backends.cuda.enable_flash_sdp(False)
    
    # generate correction
    response = model.generate(
        input_ids=tokenized_prompt["input_ids"].to(device),
        attention_mask=tokenized_prompt["attention_mask"].to(device),
        max_new_tokens=max_tokens,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # preprocess output
    # TODO: remove duplicates
    decoded_outputs = tokenizer.batch_decode(response.detach().cpu().numpy(), skip_special_tokens=True)
    text = decoded_outputs[0][len(prompt):]
    
    text_with_sentences = re.sub(r'(?<=[.!?])(?![\'"])', ' ', text)
    result = "\n".join(sent_tokenize(text_with_sentences))
    with open(result_path, 'a+') as file:
        file.write(f'# {id_str}\n')
        file.write(result)
        file.write('\n')

100%|██████████| 1/1 [01:06<00:00, 66.51s/it]


In [25]:
# from torch.utils.data import Dataset
# from transformers import AutoTokenizer

# class UAGECDataset(Dataset):
#     def __init__(
#         self, 
#         generator: object, 
#         device: str,
#         prompt: object,
#         max_sentences=None,
#         samples: int = None # if none will use all
#     ):
#         self.text_data =  generator#list(generator)
        
#         if samples:
#             self.text_data = self.text_data[:samples]
        
#         self.max_sentences = max_sentences
#         self.device = device
#         self.prompt = prompt

#     def __len__(self):
#         return len(self.text_data)

#     def __getitem__(self, idx):
#         sample = self.text_data[idx]
        
#         inputs: str = self._preprocess_text(
#             text=sample.source, 
#             target_text=sample.target
#         )
#         encodings = self._tokenize_text(
#             text=inputs,
#         ).to(self.device)

#         return {
#             'prompt': inputs,
#             'input_ids': encodings["input_ids"].squeeze(0),
#             'attention_mask': encodings["attention_mask"].squeeze(0),
#         }
    
#     def _preprocess_text(self, text: str, target_text: str) -> torch.tensor:
#         # Select top n sentences
#         text = "".join(sent_tokenize(text)[:self.max_sentences] if self.max_sentences else sent_tokenize(text))
#         target_text = "".join(sent_tokenize(target_text)[:self.max_sentences] if self.max_sentences else sent_tokenize(target_text))
#         # Add instructions (prepend prompt)
#         text = self._format_prompt(text=text)

#         text = self._normalize_spaces(text=text)
#         target_text = self._normalize_spaces(text=target_text)
#         # Add target response to input text
#         text += target_text
        
#         return text
    
#     def _format_prompt(self, text: str) -> str:
#         return self.prompt.format_prompt(
#             query=text,
#             error_types=ErrorConstants.ERROR_TYPES
#         ).to_string()
    
#     def _tokenize_text(self, text: str):
#         return tokenizer(
#             text, 
#             max_length=tokenizer_max_len, 
#             padding="max_length", 
#             truncation=True, 
#             return_tensors="pt"
#         )
    
#     def _add_target(self, text: str, target_text: str):
#         return self.tokenizer(
#             text, 
#             max_length=self.tokenizer_max_len, 
#             padding="max_length", 
#             truncation=True, 
#             return_tensors="pt"
#         )
    
#     def _normalize_spaces(self, text):
#         return ' '.join(text.split())

In [26]:
# import gc

# def clear_gpu_memory():
#     torch.cuda.empty_cache()
#     print(gc.collect())

In [27]:
# clear_gpu_memory()

In [28]:
# import time
# from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo

# def wait_until_enough_gpu_memory(min_memory_available, max_retries=10, sleep_time=5):
#     nvmlInit()
#     handle = nvmlDeviceGetHandleByIndex(torch.cuda.current_device())

#     for _ in range(max_retries):
#         clear_gpu_memory()
#         info = nvmlDeviceGetMemoryInfo(handle)
#         if info.free >= min_memory_available:
#             break
#         print(f"Waiting for {min_memory_available} bytes of free GPU memory. Retrying in {sleep_time} seconds...")
#         time.sleep(sleep_time)
#     else:
#         raise RuntimeError(f"Failed to acquire {min_memory_available} bytes of free GPU memory after {max_retries} retries.")

# # Usage example
# min_memory_available = 2 * 1024 * 1024 * 1024  # 2GB
# clear_gpu_memory()
# wait_until_enough_gpu_memory(min_memory_available)